<a href="https://colab.research.google.com/github/omarramy-74/RAG/blob/main/RAG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_community
%pip install langchain-together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 2.2 MB/s eta 0:00:00


In [28]:
import pandas as pd
import numpy as np
from langchain_together import ChatTogether
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain_core.messages import SystemMessage
from pydantic import BaseModel
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain

In [3]:
model = ChatTogether(model = "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
                     api_key = "tgp_v1_frWMlsoybz07FVkKvUwHnvuekBp2yQ7u7BX4Kl5bjd0")

In [11]:
book = "FinalOmarCv.pdf"

In [14]:
loader = PyPDFLoader(book)
books = loader.load()

In [15]:
rc_spliter = RecursiveCharacterTextSplitter(
    separators= ["\n\n","\n","."," "],
    chunk_size=1000,
    chunk_overlap=100,
)
docs = rc_spliter.split_documents(books)


In [27]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2") # Explicitly set model_name

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
vector_store = Chroma.from_documents(docs, embeddings)

In [20]:
Retrival = vector_store.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k":2}
)

In [23]:
template = ChatPromptTemplate.from_messages(
    [
    SystemMessage(content = "You Are a pdf reader that when human asks you something in his uploaded cv you retrive info using rag"),
    ("system", "You have to make sure that the pdf are read correctly"),
    ("system","you have to make sure to make the retrived info a good situation"),
    ("human","find this info in the book {info}"),
    ("ai","Here are the answer {answer}")
]
)

In [36]:
def Rag(Question):
  retrieved_docs = Retrival.invoke(Question)
  formatted_input = {"info": Question, "answer": retrieved_docs}
  Rag_chain = template | model
  return Rag_chain.invoke(formatted_input)

In [35]:
print(Rag("What is the Objective?").content)

The objective is: 

"Motivated Computer Science student at Ain Shams University with a strong foundation in machine learning, data analysis, and Python programming. Eager to apply practical knowledge of classification, regression, and anomaly detection in a real-world setting. Seeking a Machine Learning Internship at Valify Solutions to contribute to innovative AI-driven solutions while gaining hands-on experience in model development, evaluation, and deployment."
